In [1]:
from __future__ import print_function
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:90% !important; } </style>"))
import matplotlib.pyplot as plt
from pkg.marker_config import *
from pkg.constraint_graph import *
from pkg.constraint.constraint_action import *
from pkg.constraint.constraint_object import *
from pkg.constants import *
from pkg.utils.plot_utils import *
from pkg.utils.utils import *
from pkg.environment_builder import *
from pkg.ui.ui_broker import *
from pkg.controller.combined_robot import *
from pkg.controller.combined_robot import CombinedRobot, XYZ_RPY_ROBOTS_DEFAULT
from pkg.data_collecting.sampling import *

VISUALIZE = True
graph = None
SAMPLED_DATA = defaultdict(dict)
UPDATE_DAT = True

GLOBAL_FILENAME = "global.json"
WORLD_FILENAME = "world.json"
SCENE_FILENAME = "scene.json"
DATA_PATH = "./data"
CONVERTED_PATH = "./data/converted"
try_mkdir(CONVERTED_PATH)

gtimer = GlobalTimer.instance()
elog = Logger()
crob = CombinedRobot(connection_list=(False, False))

CHECK_DICT = {}

# ## Load Global params
DATASET_LIST = sorted(filter(lambda x: x not in ["backup", "converted"], os.listdir(DATA_PATH)))

connection_list
(False, False)


## dataset

In [2]:
DATASET = DATASET_LIST[0]
CHECK_DICT[DATASET] = {}
CURRENT_PATH = os.path.join(DATA_PATH, DATASET)
try_mkdir(os.path.join(CONVERTED_PATH, DATASET))


## Load global params
GLOBAL_PARAMS = load_json(os.path.join(CURRENT_PATH, GLOBAL_FILENAME))
WDH = GLOBAL_PARAMS["WDH"]
L_CELL = GLOBAL_PARAMS["L_CELL"]
RATIO_DIMS = GLOBAL_PARAMS["RATIO_DIMS"]
REACH_OFFSET_DICT = GLOBAL_PARAMS["REACH_OFFSET_DICT"]
GRIPPER_REFS = GLOBAL_PARAMS["GRIPPER_REFS"]
BASE_LINK = GLOBAL_PARAMS["BASE_LINK"]
S_F_RATIO = GLOBAL_PARAMS["S_F_RATIO"]
TIMEOUT = GLOBAL_PARAMS["TIMEOUT"]

CENTER = tuple(np.divide(WDH, 2, dtype=np.float))
Ws, Ds, Hs = WDH
Nw, Nd, Nh = Nwdh = int(Ws / L_CELL), int(Ds / L_CELL), int(Hs / L_CELL)
L_MAX = L_CELL * RATIO_DIMS
print("scene size: {} ({},{},{})".format(Nw * Nd * Nh, Nw, Nd, Nh))


scene size: 3375 (15,15,15)


## world

In [ ]:
#Iterate world
WORLD_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(CURRENT_PATH)))

for WORLD in WORLD_LIST:
    CHECK_DICT[DATASET][WORLD] = {}
    WORLD_PATH = os.path.join(CURRENT_PATH, WORLD)
    try_mkdir(os.path.join(CONVERTED_PATH, DATASET, WORLD))
    SAMPLED_DATA["WORLD"] = load_json(os.path.join(WORLD_PATH, WORLD_FILENAME))
    Trbt_dict = SAMPLED_DATA["WORLD"]["Trbt_dict"]
    reach_center_dict = {k: tuple(np.add(v[0], REACH_OFFSET_DICT[k])) for k, v in Trbt_dict.items()}

    cam = None
    # set urdf
    xcustom, JOINT_NAMES, LINK_NAMES, urdf_content = set_custom_robots(crob.robots_on_scene, Trbt_dict,
                                                                       crob.custom_limits, start_rviz=VISUALIZE)
    ghnd = GeometryHandle(urdf_content)
    time.sleep(2)

    # set graph
    graph = ConstraintGraph(ghnd=ghnd, urdf_path=URDF_PATH, joint_names=JOINT_NAMES, link_names=LINK_NAMES,
                            urdf_content=urdf_content, combined_robot=crob)
    graph.set_camera(cam)
    graph.set_cam_robot_collision(_add_cam_poles=False, color=(1, 1, 0, 0.3))
    if VISUALIZE: graph.set_rviz()

    # start UI
    ui_broker = UIBroker.instance()
    ui_broker.initialize(graph)
    ui_broker.start_server()
    
    ui_broker.set_tables()

    for gripper in GRIPPER_REFS.values():
        graph.register_binder(name=gripper['bname'], _type=FramedTool, point=gripper['tcp_ref'], rpy=[0, 0, 0],
                              link_name=gripper['link_name'])
    graph.register_binder(name='base', _type=PlaceFrame, point=[0, 0, 0], rpy=[0, 0, 0], link_name=BASE_LINK)
    vtem = graph.ghnd.create_safe(name="virtual", gtype=GEOTYPE.SPHERE, link_name=BASE_LINK,
                                  dims=(0, 0, 0), center=(0, 0, 0), rpy=(0, 0, 0), collision=False, display=False
                                  )
    graph.add_object("virtual",
                     SingleHandleObject(_object=vtem,
                                        action_point=FramePoint(name="point", _object=vtem, point=(0, 0, 0),
                                                                rpy=(0, 0, 0), name_full=None)),
                     binding=("point", "base"))

    obj_list = []
    col_obj_list = []

    # Iterate scene
    SCENE_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(WORLD_PATH)))
    for SCENE in SCENE_LIST:
        try_mkdir(os.path.join(CONVERTED_PATH, DATASET, WORLD, SCENE))
        CHECK_DICT[DATASET][WORLD][SCENE] = {}
        SCENE_PATH = os.path.join(WORLD_PATH, SCENE)
        SAMPLED_DATA["SCENE"] = load_json(os.path.join(SCENE_PATH, SCENE_FILENAME))
        Q_s = np.array(SAMPLED_DATA["SCENE"]["Q_s"])
        Q_s, links, link_verts, link_ctems, link_rads = sample_joint(graph, Q_s_loaded=Q_s)
        Q_s_dict = SAMPLED_DATA["SCENE"]["Q_s_dict"]
        obj_dat = SAMPLED_DATA["SCENE"]["obj_dat"]

        if VISUALIZE:
            graph.show_pose(Q_s)
            time.sleep(1)
            graph.show_pose(Q_s)
            time.sleep(1)
            graph.show_pose(Q_s)
        for obj in obj_list: graph.remove_geometry(obj)
        for odat in obj_dat:
            nbox, gtype, dims, color, center, rpy = odat["nbox"], getattr(GEOTYPE, odat["gtype"]), odat["dims"], \
                                                    odat["color"], odat["center"], odat["rpy"]
            obj = graph.ghnd.create_safe(
                name="{}_{}_{}_{}".format(gtype.name, *nbox), link_name=BASE_LINK, gtype=gtype,
                center=center, rpy=rpy, dims=dims, color=color, display=True, collision=True, fixed=True)
            obj_list.append(obj)
            graph.add_marker(obj, vis=VISUALIZE)

        for obj in col_obj_list: graph.remove_geometry(obj)

        if VISUALIZE: graph.set_rviz()

        # Iterate actions
        ACTION_LIST = sorted(filter(lambda x: x != SCENE_FILENAME, os.listdir(SCENE_PATH)))
        for ACTION in ACTION_LIST:
            N_action = get_action_count(CONVERTED_PATH, DATASET, WORLD, SCENE, ACTION)
            print("[BEGIN] {} - {} - {} - {} ===============".format(DATASET, WORLD, SCENE, ACTION))
            CHECK_DICT[DATASET][WORLD][SCENE][ACTION] = {}
            snode_dict = {int(k): v for k, v in load_json(os.path.join(SCENE_PATH, ACTION)).items()}
            snode_keys = sorted(snode_dict.keys())

            if snode_keys:
                sk0 = snode_keys[0]
                rname, tar, inhand_coll, obj_coll = \
                    snode_dict[sk0]["rname1"], snode_dict[sk0]["rname2"], \
                    snode_dict[sk0]["obj1"]["collision"], snode_dict[sk0]["obj2"]["collision"]
            else:
                print("[END] {} - {} - {} - {} ===============".format(DATASET, WORLD, SCENE, ACTION))
                print("No Item")


            if rname and tar:  # handover case
                action_type = "HANDOVER"
            elif inhand_coll:  # place case
                action_type = "PLACE"
            elif obj_coll:  # pick case
                action_type = "PICK"
            else:
                raise (RuntimeError("non-implemented case"))

            ID=None

            elog = Logger()
            acquired = False
            for i_act in range(N_action):
                scene_data, success, skey = load_scene_data(CONVERTED_PATH, DATASET, WORLD, SCENE, ACTION, i_act, crob.joint_num)
                try:
                    snode = snode_dict[skey]
                    rname, inhand, obj, tar, dims_bak, color_bak, succ, _ = load_manipulation_from_dict(snode,
                                                                                                        graph.ghnd)
                except Exception as e:
                    if not elog.log(str(e)):
                        break
                    continue
                try:

                    if rname and tar:  # handover case
                        remove_map = [[], [0, 1]]
                        action_type = "HANDOVER"
                    elif inhand.collision:  # place case
                        remove_map = [[], [0, 1]]
                        action_type = "PLACE"
                    elif obj.collision:  # pick case
                        remove_map = [[1], [0]]
                        action_type = "PICK"
                    else:
                        remove_map = [[], [0,1]]
                        action_type = "None"
                        raise (RuntimeError("non-implemented case"))
                    if VISUALIZE: show_manip_coords(graph, GRIPPER_REFS, action_type, rname, inhand, obj, rname2=tar)
                    raw_input("Press Enter to continue...")
                except Exception as e:
                    if not elog.log(str(e)):
                        break
                finally:
                    remove1 = [[inhand, obj][iii] for iii in remove_map[0]]
                    remove2 = [[inhand, obj][iii] for iii in remove_map[1]]
                    reset_rendering(graph, action_type, remove1, remove2, dims_bak, color_bak, sleep=True,
                                    vis=VISUALIZE)
            print("[END] {} - {} - {} - {} ===============".format(DATASET, WORLD, SCENE, ACTION))
        print("============================ TERMINATE {} ===================================".format(ID))

Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
[BEGIN] 20201208-121454 - WORLD-20201208-121454 - SCENE-20201208-121456 - 20201208-121507.json ===============
Press Enter to continue...
Press Enter to continue...
Press Enter to continue...
Press Enter to continue...
[END] 20201208-121454 - WORLD-20201208-121454 - SCENE-20201208-121456 - 20201208-121507.json ===============
[BEGIN] 20201208-121454 - WORLD-20201208-121454 - SCENE-20201208-121456 - 20201208-121519.json ===============
Press Enter to continue...
Press Enter to continue...
Press Enter to continue...
Press Enter to continue...
[END] 20201208-121454 - WORLD-20201208-121454 - SCENE-20201208-121456 - 20201208-121519.json ===============
[BEGIN]